## Challenge 1: Understanding the Input Data

1. Why do we use split our data into train/validation/test?
2. What is the shape of our input data partitions?
3. What is the type of the data?

**BONUS:**

4. What is the distribution of the target classes within the data, is it balanced?

In [ ]:
# 1.1 Why do we use split our data into train/validation/test?

# We need to train our model and avoid overfitting.

# We use the training set to fit the model.
# We use the validation set to tune the hyperparameters of our model.
# We use the holdout test set to determine our final performance. 

In [ ]:
# 1.2 What is the shape of our input data partitions?

# The training set contains 5000 examples
# The validation set contains 1000 examples 
# The test set contains 1000 examples
# The X are 3-dimensional
# The y are 1 dimensional

In [ ]:
# 1.3
[type(partition) for partition in [x_train, y_train, x_val, y_val, x_test, y_test]]
# All are numpy arrays

In [ ]:
# BONUS 1.4

# Simply plot the histogram of each y in a different cell, for example
# plt.hist(y_train)
# plt.title('Train Class Distribution');
# repeat for y_val and y_test
 
# or make subplots
def plot_target_distributions(targets, titles):
    """
    Returns the distribution of target classes.
    """
    
    fig, axes = plt.subplots(3,1, figsize = (10,10))
    
    for ax, target, title in zip(axes, targets, titles):
        ax.hist(target) 
        ax.set_title(f"{title} Class Distribution")
    
    return plt.show()

plot_target_distributions([y_train, y_val, y_test], ["Train", "Validation", "Test"])
    

## Challenge 2: Build your own neural network

1. Build and compile your own neural network in an object called `my_network`. Feel free to choose your own:
    - Architecture
    - Activation Function
    - Epochs
    
2. Train your model, saving the results to an object called `history_my_network`.

In [ ]:
# 2.1
# An example network with: 3 dense layers, each with 512 neurons and a dropout of 0.3

my_network = Sequential()
my_network.add(Dense(512, activation= "relu", input_shape=(28*28,)))
my_network.add(Dropout(0.3))
my_network.add(Dense(512, activation= "relu"))
my_network.add(Dropout(0.3))
my_network.add(Dense(512, activation= "relu"))
my_network.add(Dropout(0.3))
my_network.add(Dense(10, activation="softmax"))

my_network.compile(optimizer = 'rmsprop', 
                     loss = 'categorical_crossentropy',
                     metrics = ['accuracy'])

In [ ]:
# 2.2
# Train model for 20 epochs with batch size of 128
history_my_network = my_network.fit(x_train_trans, 
                            y_train_trans, 
                            epochs=20, 
                            batch_size=128, 
                            validation_data=(x_val_trans, y_val_trans))


## Challenge 3: Evaluate your own model

Use your own model from challenge 2 to evaluate its general performance.

1. Visualize the training and validation accuracy over each epoch.
2. Print the accuracy of your model on the test set.

In [ ]:
# 3.1 
plot_epoch_accuracy(history_my_network.history)

In [ ]:
# 3.2
get_model_accuracy(my_network, x_test_trans, y_test_trans)

## Challenge 4: Save your own model

Save your own model `my_network` in the same directory we saved our other models in.

In [ ]:
# 4.1
save_models(my_network, "my_network")